In [1]:
import keras
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Dropout, Input, Flatten
from keras.regularizers import l2, activity_l2,l1
from keras.utils import np_utils
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from keras.applications.vgg16 import preprocess_input
from PIL import Image
from scipy import misc
from keras.optimizers import SGD
# from keras.utils.visualize_util import plot
from os import listdir
import numpy as np
import matplotlib.pyplot as plt
import scipy

Using Theano backend.
Using gpu device 1: Tesla K20c (CNMeM is disabled, cuDNN 4007)


In [44]:
def getModel( output_dim ):
    # output_dim: the number of classes (int)
    # return: compiled model (keras.engine.training.Model)
    
    vgg_model = VGG16( weights='imagenet', include_top=True )
    vgg_out = vgg_model.layers[-1].output 
#     vgg_out = Flatten()(vgg_out)
    
    vgg_out = Dropout(0.25)(vgg_out)
    #Create new transfer learning model    
    out = Dense( output_dim, activation="softmax", W_regularizer = l2(0.2))( vgg_out )

    tl_model = Model( input=vgg_model.input, output=out )
    for layer in tl_model.layers[0:-1]:
        layer.trainable = False            

    tl_model.compile(loss= "categorical_crossentropy", optimizer="adagrad", metrics=["acc"])
    print tl_model.summary()
    return tl_model

In [3]:
# define functions to laod images
def loadBatchImages(path,s, nVal = 2):
    # return array of images
    catList = listdir(path)
    loadedImagesTrain = []
    loadedLabelsTrain = []
    loadedImagesVal = []
    loadedLabelsVal = []


    for cat in catList[0:256]:
        deepPath = path+cat+"/"
        # if cat == ".DS_Store": continue
        imageList = listdir(deepPath)
        indx = 0
        for images in imageList[0:s + nVal]:                
            img = load_img(deepPath + images)
            img = img_to_array(img)
            img = misc.imresize(img, (224,224))
            img = scipy.misc.imrotate(img,180)
            if indx < s:
                loadedLabelsTrain.append(int(images[0:3])-1)
                loadedImagesTrain.append(img)
            else:
                loadedLabelsVal.append(int(images[0:3])-1)
                loadedImagesVal.append(img)
            indx += 1
            
#     return np.asarray(loadedImages), np.asarray(loadedLabels)
    return loadedImagesTrain, np_utils.to_categorical(loadedLabelsTrain), loadedImagesVal, np_utils.to_categorical(loadedLabelsVal) 
    
def shuffledSet(a, b):
    # shuffle the entire dataset
    assert np.shape(a)[0] == np.shape(b)[0]
    p = np.random.permutation(np.shape(a)[0])
    return (a[p], b[p])


In [4]:
path = "/mnt/cube/VGG_/256_ObjectCategories/"    
samCat = 32 # number of samples per category

data, labels, dataVal, labelsVal = loadBatchImages(path,samCat, nVal = 2)

data = preprocess_input(np.float64(data))
data = data.swapaxes(1, 3).swapaxes(2, 3)

dataVal = preprocess_input(np.float64(dataVal))
dataVal = dataVal.swapaxes(1, 3).swapaxes(2, 3)

train = shuffledSet(np.asarray(data),labels)
val = shuffledSet(np.asarray(dataVal),labelsVal)

In [5]:
plt.imshow(train[0][0][0])
plt.show()
print train[0].shape, val[0].shape

(8192, 3, 224, 224) (512, 3, 224, 224)


In [45]:
output_dim = 256
tl_model = getModel(output_dim) 

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 224, 224)  1792        input_3[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, 224, 224)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 112, 112)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [1]:
nb_epoch = 5

history = tl_model.fit(train[0], train[1], batch_size = 64, nb_epoch = nb_epoch, validation_data = val, 
                       shuffle = True)

keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0, patience = 2, verbose = 0, mode='auto')

In [3]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss for %d samples/ int' % samCat)
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['val_acc'])
plt.title('model accuracy for %d samples per category' % samCat)
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.show()

In [23]:
tl_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 3, 224, 224)   0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, 64, 224, 224)  1792        input_2[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, 64, 224, 224)  36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, 64, 112, 112)  0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [26]:
plt.imshow(tl_model.layers[-8].get_weights()[0][0], cmap = 'gray')
plt.show()

In [35]:
## Reference: https://blog.keras.io/how-convolutional-neural-networks-see-the-world.html
from keras import backend as K

layer_name = 'conv5_1'
filter_index = 0  # can be any integer from 0 to 511, as there are 512 filters in that layer

# build a loss function that maximizes the activation
# of the nth filter of the layer considered
layer_output = tl_model.layers[-].output
loss = K.mean(layer_output[:, filter_index, :, :])
input_img = tl_model.layers[0].output
# compute the gradient of the input picture wrt this loss
grads = K.gradients(loss, input_img)[0]

# normalization trick: we normalize the gradient
grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)

# this function returns the loss and grads given the input picture
iterate = K.function([input_img], [loss, grads])

In [40]:
# we start from a gray image with some noise
input_img_data = np.random.random((1, 3, 224, 224)) * 20 + 128.
step = 0.1
# run gradient ascent for 20 steps
for i in range(500):
    loss_value, grads_value = iterate([input_img_data])
    input_img_data += grads_value * step

In [42]:
def deprocess_image(x):
    # normalize tensor: center on 0., ensure std is 0.1
    x -= x.mean()
    x /= (x.std() + 1e-5)
    x *= 0.1

    # clip to [0, 1]
    x += 0.5
    x = np.clip(x, 0, 1)

    # convert to RGB array
    x *= 255
    x = x.transpose((1, 2, 0))
    x = np.clip(x, 0, 255).astype('uint8')
    return x

img = input_img_data[0]
img = deprocess_image(img)
# imshow('%s_filter_%d.png' % (layer_name, filter_index), img)
plt.imshow(img)
plt.show()